# Retrieval

In [ ]:
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
caminhos = [
    "arquivos/Explorando o Universo das IAs com Hugging Face.pdf",
    "arquivos/Explorando a API da OpenAI.pdf",
    "arquivos/Explorando a API da OpenAI.pdf",
    ]

paginas = []

for caminho in caminhos:
    loader = PyPDFLoader(caminho)
    paginas.extend(loader.load())

recur_split = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=["\n\n", "\n", ".", " ", ""]
)

documents = recur_split.split_documents(paginas)

### Modificando metadata

In [ ]:
for i, doc in enumerate(documents):
    doc.metadata['source'] = doc.metadata['source'].replace('arquivos/', '')
    doc.metadata['doc_id'] = i

In [ ]:
documents[2].metadata

In [ ]:
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.chroma import Chroma

embeddings_model = OpenAIEmbeddings()

### Criando VectorStore

In [ ]:
diretorio = 'arquivos/chroma_retrival_bd'

vectordb = Chroma.from_documents(
    documents=documents,
    embedding=embeddings_model,
    persist_directory=diretorio
)

## Semantic Search

In [ ]:
pergunta = 'O que é a OpenAI?'

docs = vectordb.similarity_search(pergunta, k=3)
for doc in docs:
    print(doc.page_content)
    print(f'==========={doc.metadata}\n\n')


### Limitações do semantic search

#### Trechos com muito alta similaridade e pouca infomação adicional

In [ ]:
pergunta = 'O que é a OpenAI?'

docs = vectordb.similarity_search(pergunta, k=3)
for doc in docs:
    print(doc.page_content)
    print(f'==========={doc.metadata}\n\n')

In [ ]:
pergunta = 'O que a apostila de Hugging Face fala sobre a OpenAI e o ChatGPT?'

docs = vectordb.similarity_search(pergunta, k=3)
for doc in docs:
    print(doc.page_content)
    print(f'==========={doc.metadata}\n\n')

## MMR

In [ ]:
pergunta = 'O que é a OpenAI?'

docs = vectordb.max_marginal_relevance_search(pergunta, k=3, fetch_k=10)
for doc in docs:
    print(doc.page_content)
    print(f'==========={doc.metadata}\n\n')

## Filtragem

In [ ]:
pergunta = 'O que a apostila de Hugging Face fala sobre a OpenAI e o ChatGPT?'

docs = vectordb.similarity_search(
    pergunta, 
    k=3,
    filter={'source': 'Explorando o Universo das IAs com Hugging Face.pdf'})
for doc in docs:
    print(doc.page_content)
    print(f'==========={doc.metadata}\n\n')

In [ ]:
pergunta = 'O que a apostila de Hugging Face fala sobre a OpenAI e o ChatGPT?'

docs = vectordb.similarity_search(
    pergunta, 
    k=3,
    filter={'source': {'$in': ['Explorando o Universo das IAs com Hugging Face.pdf']}})
for doc in docs:
    print(doc.page_content)
    print(f'==========={doc.metadata}\n\n')

In [ ]:
pergunta = 'O que a apostila de Hugging Face fala sobre a OpenAI e o ChatGPT?'

docs = vectordb.similarity_search(
    pergunta, 
    k=3,
    filter={'$and':
            [{'source': {'$in': ['Explorando o Universo das IAs com Hugging Face.pdf']}},
            {'page': {'$in': [10, 11, 12, 13]}}],
            })
for doc in docs:
    print(doc.page_content)
    print(f'==========={doc.metadata}\n\n')

## LLM Aided Retrieval

In [ ]:
from langchain_openai.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.schema import AttributeInfo

In [ ]:
metadata_info = [
    AttributeInfo(
        name='source',
        description='Nome da apostila de onde o texto original foi retirado. Deve ter o valor de: \
            Explorando o Universo das IAs com Hugging Face.pdf ou Explorando a API da OpenAI.pdf',
        type='string'
    ),
    AttributeInfo(
        name='page',
        description='A página da apostila de onde o texto se origina',
        type='integer'
    ),

]

In [ ]:
document_description = 'Apostilas de cursos'
llm = OpenAI()
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectordb,
    document_description,
    metadata_info,
    verbose=True
)

In [ ]:
pergunta = 'O que a apostila de Hugging Face fala sobre a OpenAI e o ChatGPT?'

docs = retriever.invoke(pergunta)
for doc in docs:
    print(doc.page_content)
    print(f'==========={doc.metadata}\n\n')

In [ ]:
pergunta = 'Quais detalhes são descritos na página 44 da apostila Explorando a API da OpenAI?'

docs = retriever.invoke(pergunta)
for doc in docs:
    print(doc.page_content)
    print(f'==========={doc.metadata}\n\n')

In [ ]:
from langchain.globals import set_debug

set_debug(True)

pergunta = 'Quais detalhes são descritos na página 44 da apostila Explorando a API da OpenAI?'

docs = retriever.invoke(pergunta)
for doc in docs:
    print(doc.page_content)
    print(f'==========={doc.metadata}\n\n')


In [ ]:
import json

_json = '{\n    \"query\": \"detalhes\",\n    \"filter\": \"and(eq(\\\"source\\\", \\\"Explorando a API da OpenAI.pdf\\\"), eq(\\\"page\\\", 44))\"\n}'
print(json.dumps(json.loads(_json), indent=4))

